# First Assignment - Explore and cluster the neighborhoods in Toronto

## Scraping data from Wiki

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(url)
html = page.read().decode("utf-8")
toronto_soup = BeautifulSoup(html, "html.parser")

## Creating DataFrame

In [ ]:
table = toronto_soup.find('table', attrs={'class':'wikitable sortable'}) #getting the postal code tables
table_rows = table.find_all('tr')
headers = [h.get_text().rstrip() for h in table.find_all('th')] #getting the headers
l = []
for tr in table_rows: #retrieving the values
    td = tr.find_all('td')
    if td:
        row = [tr.text.rstrip() for tr in td]
        l.append(row)
df = pd.DataFrame(l, columns=headers)
df.dropna()
print("DataFrame original shape: ", df.shape)
df.head()

## Dropping 'Not Assigned' borough cells

In [ ]:
###################################
# Dropping 'Not Assigned' cells ###
###################################
df_filtered = df[df['Borough'] != 'Not assigned'] 
print("DataFrame filtered shape: ", df_filtered.shape)
df_filtered.head()

## Checking for duplicates Postal Codes

In [ ]:
#############################
# Checking for duplicates ###
#############################
duplicate = df_filtered.duplicated(subset=['Postal Code'])
duplicate_values = 0
if duplicate.any():
    duplicate_values += 1
    print(df_filtered['Postal Code'][~duplicate], end='\n\n')
if duplicate_values == 0:
    print("No duplicate values any longer") #there are no duplicates
    

## Looking for 'Not Assigned' Neighbourhood

In [ ]:
print(df_filtered.loc[df_filtered['Neighbourhood'] == 'Not Assigned'])

## Dataframe shape

In [ ]:
print("Final data frame shape:",  df_filtered.shape)

# Second Assignment - Getting Longitude and Latitude of each postal code

In [ ]:
#Installing geocoder

import sys
#!conda install --yes --prefix {sys.prefix} geocoder
!{sys.executable} -m pip install geocoder

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
coordinates = []
#######################################
#### NOT WORKING: PERMISSION DENIED ###
#######################################
for postal_code in df_filtered:
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('Toronto, Ontario', sensor=False)
        lat_lng_coords = g.latlng
    coordinates = [c for c in lat_lng_coords]


In [35]:
#Loading data from csv file
df_coordinates = pd.read_csv('https://raw.githubusercontent.com/vscoca/Coursera_Capstone/main/Geospatial_Coordinates.csv')  
df_coordinates.head()
df_final = df_filtered.join(df_coordinates.set_index('Postal Code'), on='Postal Code')
df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
